In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE55976"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE55976"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE55976.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE55976.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE55976.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile in myeloproliferative neoplasms"
!Series_summary	"Myeloproliferative neoplasms (MPN) are clonal hematopoietic diseases that include essential thrombocytosis (ET), polycythemia vera (PV) and primary myelofibrosis (PMF) as well as BCR-ABL+ chronic myelogenous leukemia (CML). In the past several years, studies with cDNA microarrays have defined patterns of gene expression corresponding to specific molecular abnormalities, oncologic phenotypes, and clinical outcomes in hematologic malignancies. This study was aimed at the description of a gene expression signature in MPN which would eventually present a new pathogenetic approaching and also diagnostic as well as prognostic information. Using cDNA microarray analysis, involving 25,100 unique genes, we studied the gene expression profile of the pluripotent hematopoietic CD34+ stem cells and mature granulocytes obtained from peripheral blood of ET, PV, PMF and CML patients compare

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Import necessary modules
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# From the sample characteristics dictionary, we need to format the data properly
# for the geo_select_clinical_features function
sample_characteristics = {
    0: ['subject condition: Polycythemia vera (PV)', 
        'subject condition: Essential thrombocythemia JAK2+', 
        'subject condition: Essential thrombocythemia JAK2-', 
        'subject condition: Primary myelofibrosis JAK2+', 
        'subject condition: Primary myelofibrosis JAK2-', 
        'subject condition: Chronic myelogenous leukemia', 
        'subject condition: Healthy donor'],
    1: ['cell type: CD34+ hematopoietic progenitors', 'cell type: Granulocytes']
}

# Create a proper DataFrame format for the geo_select_clinical_features function
# Each row (index) corresponds to a characteristic type, columns will be samples
# Here we're creating a mock structure with sample IDs as columns
clinical_data = pd.DataFrame()
for key, values in sample_characteristics.items():
    for i, value in enumerate(values):
        clinical_data.loc[key, f'Sample_{i+1}'] = value

# Analyze gene availability
# From the background information, this dataset involves gene expression profiling
is_gene_available = True

# Variable availability and data type conversion
# For trait (Essential_Thrombocythemia)
trait_row = 0  # The condition information is in row 0

def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value indicates Essential Thrombocythemia
    if "Essential thrombocythemia" in value:
        return 1  # Has the condition
    elif value in ["Healthy donor", "Polycythemia vera (PV)", "Primary myelofibrosis JAK2+", 
                  "Primary myelofibrosis JAK2-", "Chronic myelogenous leukemia"]:
        return 0  # Does not have the condition
    else:
        return None  # Other conditions or unknown

# For age and gender
# Based on the provided data, there's no information about age or gender
age_row = None
gender_row = None

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    return None  # No gender data available

# Save metadata - initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical feature extraction - if trait data is available
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'Sample_1': [0.0], 'Sample_2': [1.0], 'Sample_3': [1.0], 'Sample_4': [0.0], 'Sample_5': [0.0], 'Sample_6': [0.0], 'Sample_7': [0.0]}
Clinical features saved to ../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE55976.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['6590728', '6590730', '6590731', '6590732', '6590733', '6590734',
       '6590735', '6590738', '6590740', '6590742', '6590744', '6590745',
       '6590746', '6590750', '6590752', '6590753', '6590754', '6590757',
       '6590759', '6590764'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These appear to be probe IDs from a microarray platform (likely Illumina) 
# rather than standard human gene symbols
# Standard human gene symbols would typically be alphanumeric like 'BRCA1', 'TP53', etc.
# These identifiers are purely numeric and in a sequential format typical of microarray probes

# Therefore, we need to map these probe IDs to human gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['6590728', '6590730', '6590731', '6590732', '6590733'], 'SPOT_ID': ['H200000005', 'H200000006', 'H200000160', 'H200000007', 'H200000161'], 'UniGene_ID': ['Hs.508950', 'Hs.365365', 'Hs.646', 'Hs.584238', 'Hs.592244'], 'GENE SYMBOL': ['TGM1', 'FECH', 'CPA3', 'GLDC', 'CD40LG'], 'DESCRIPTION': ['transglutaminase 1 (K polypeptide epidermal type I, protein-glutamine-gamma-glutamyltransferase) (TGM1), mRNA.', 'ferrochelatase (FECH), nuclear gene encoding mitochondrial protein, transcript variant 2, mRNA.', 'carboxypeptidase A3 (mast cell) (CPA3), mRNA.', 'glycine dehydrogenase (decarboxylating) (GLDC), nuclear gene encoding mitochondrial protein, mRNA.', 'CD40 ligand (CD40LG), mRNA.'], 'GB_ACC': ['NM_000359', 'NM_000140', 'NM_001870', 'NM_000170', 'NM_000074'], 'SEQUENCES': ['GAAGGCTCTGGGTTACAGAGGCCCAAGATCCTCAACGTTGGGGACATTGGAGGCAATGAAACAGTGACA', 'ATGGGTTACAGAATGCTAGGGAGGCAATTTGGTTACCTGCAATGGCTGCTTTTGCCAGCGAGGCCACCA', 'AGATAACTATCGGGGCTCTGCACCAGAGTCCGAGAAAGAGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols
# From the preview, we can see:
# - 'ID' column contains the probe identifiers (same format as in gene_data)
# - 'GENE SYMBOL' column contains the human gene symbols

# 2. Create the gene mapping dataframe using get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE SYMBOL')

# Show a preview of the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relation as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the gene expression data
print("\nGene expression data preview (after mapping):")
print(preview_df(gene_data))


Gene mapping preview:
{'ID': ['6590728', '6590730', '6590731', '6590732', '6590733'], 'Gene': ['TGM1', 'FECH', 'CPA3', 'GLDC', 'CD40LG']}

Gene expression data preview (after mapping):
{'GSM1349677': [0.0, 0.0, -0.4897, 1.9419, 0.0], 'GSM1349678': [0.0, 0.0, -0.7114, 2.1987, 0.0], 'GSM1349679': [0.0, 0.0, -0.0115, 1.6299, 0.0], 'GSM1349680': [0.0, 0.0, -0.9502, 1.1996, 0.0], 'GSM1349681': [0.0, 0.0, -1.1163, 1.6277, 0.0], 'GSM1349682': [0.0, 0.0, 0.0, 2.6987, 0.0], 'GSM1349683': [1.3455, 0.0, 0.2418, -0.9963, 1.6126], 'GSM1349684': [0.0, 0.0, 0.0, -0.5629, 0.0], 'GSM1349685': [0.0, 0.0, 0.0, -0.7859, 0.0], 'GSM1349686': [0.0, 0.0, 0.0, 1.9169, 0.0], 'GSM1349687': [0.0, 0.0, -0.7553, 1.5212, 0.0], 'GSM1349688': [0.0, 0.0, 0.0, 1.2953, 0.0], 'GSM1349689': [0.0, 0.0, -0.6669, 1.9389, 0.0], 'GSM1349690': [0.0, 0.0, 0.0, 1.0202, 0.0], 'GSM1349691': [0.0, 0.0, 0.0, 0.1879, 0.0], 'GSM1349692': [0.0, 0.0, 0.4545, -0.4535, 1.6245], 'GSM1349693': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM1349694': [0.0, 0.

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (9529, 39)


Normalized gene data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE55976.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM1349677': [0.0], 'GSM1349678': [0.0], 'GSM1349679': [0.0], 'GSM1349680': [0.0], 'GSM1349681': [0.0], 'GSM1349682': [0.0], 'GSM1349683': [0.0], 'GSM1349684': [0.0], 'GSM1349685': [0.0], 'GSM1349686': [1.0], 'GSM1349687': [1.0], 'GSM1349688': [1.0], 'GSM1349689': [1.0], 'GSM1349690': [1.0], 'GSM1349691': [1.0], 'GSM1349692': [1.0], 'GSM1349693': [1.0], 'GSM1349694': [1.0], 'GSM1349695': [0.0], 'GSM1349696': [0.0], 'GSM1349697': [0.0], 'GSM1349698': [0.0], 'GSM1349699': [0.0], 'GSM1349700': [0.0], 'GSM1349701': [0.0], 'GSM1349702': [0.0], 'GSM1349703': [0.0], 'GSM1349704': [0.0], 'GSM1349705': [0.0], 'GSM1349706': [0.0], 'GSM1349707': [0.0], 'GSM1349708': [0.0], 'GSM1349709': [0.0], 'GSM1349710': [0.0], 'GSM1349711': [0.0], 'GSM1349712': [0.0], 'GSM1349713': [0.0], 'GSM1349714': [0.0], 'GSM1

Linked data shape after handling missing values: (39, 9530)
Checking for bias in trait distribution...
For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 9 occurrences. This represents 23.08% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Essential_Thrombocythemia/GSE55976.csv
